In [1]:
import geopandas as gpd
import folium
from datetime import datetime, timedelta

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AdLIrYftNLr2ZuR51ROK0EiWGkyIO_NhCwxMCFog-SSbAFl6R9WyvN68Nuo



Successfully saved authorization token.


In [25]:

# Define your region of interest - Ethiopia's bounding box
ethiopia_bbox = ee.Geometry.Rectangle([32.999, 3.404, 47.982, 14.894])
niger_bbox = ee.Geometry.Rectangle([0.1666, 11.6936, 15.9956, 23.5178])
usa_bbox = ee.Geometry.Rectangle([-125.0, 24.5, -66.95, 49.0])


# Specify the year of interest
year = 2020
start_date = ee.Date(f'{year}-01-01')
end_date = ee.Date(f'{year}-12-31')

# Load the temperature dataset for the specified year and filter by the region
temp_dataset = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date) \
    .filterBounds(niger_bbox) \
    .select('maximum_2m_air_temperature')

# Define a function to identify hot days (temperatures above 35C, adjusted for Kelvin)
def is_hot_day(image):
    return image.subtract(273.15).gt(30).rename('hot_day')

# Apply the function across the dataset
hot_days = temp_dataset.map(is_hot_day)

# Aggregate the hot days into a single image representing the count of hot days in the year
annual_hot_days_count = hot_days.sum()


In [26]:

# Function to generate the map with dynamic opacity for the hot days layer
def generate_map(image, vis_params, location=[17.607789, 8.081666], zoom_start=6, opacity=0.5):
    # Generate map ID and token
    map_id_dict = image.getMapId(vis_params)
    
    # Create a Folium map object
    folium_map = folium.Map(location=location, zoom_start=zoom_start)
    
    # Add the hot days layer to the map with specified opacity
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        overlay=True,
        name='Annual Hot Days Count',
        opacity=opacity  # Set the layer opacity here
    ).add_to(folium_map)
    
    # Optionally, add a layer control widget to the map
    folium.LayerControl().add_to(folium_map)
    
    return folium_map

# Assuming `annual_hot_days_count` is the image representing the count of hot days
# Adjust vis_params as needed based on the dataset's specifics
vis_params = {
    'min': 0, 
    'max': 365,  # Example maximum value; adjust based on actual data
    'palette': ['white', 'yellow', 'orange', 'red']
}

# Example usage of the modified function
# Replace `annual_hot_days_count` with your specific image variable if different
map = generate_map(annual_hot_days_count, vis_params, opacity=0.7)
map


In [27]:
#Create legend
# Adjust vis_params as needed based on the dataset's specifics
vis_params = {
    'min': 0, 
    'max': 365,  # Example maximum value; adjust based on actual data
    'palette': ['white', 'yellow', 'orange', 'red']
}

def generate_map_with_legend(image, vis_params, location=[17.607789, 8.081666], zoom_start=6, opacity=0.5):
    # Generate map ID and token
    map_id_dict = image.getMapId(vis_params)
    
    # Create a Folium map object
    folium_map = folium.Map(location=location, zoom_start=zoom_start)
    
    # Add the hot days layer to the map with specified opacity
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        overlay=True,
        name='Annual Hot Days Count',
        opacity=opacity
    ).add_to(folium_map)
    
    # Define the HTML for a horizontal legend with specific values
    legend_html = '''
    <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 220px; height: 60px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white; padding: 10px; opacity: 0.8;">
        <b>Legend</b><br>
        <div style="background: linear-gradient(to right, white, yellow, orange, red); 
                    width: 170px; height: 20px; display: inline-block;"></div>
        <div style="width: 170px; position: relative; bottom: 30px;">
            <div style="position: absolute; left: 0;">0</div>
            <div style="position: absolute; left: 50%;">182</div>
            <div style="position: absolute; right: 0;">365</div>
        </div>
    </div>
    '''
    
    # Add the HTML to the Folium map
    folium_map.get_root().html.add_child(folium.Element(legend_html))
    
    # Optionally, add a layer control widget to the map
    folium.LayerControl().add_to(folium_map)
    
    return folium_map

# Example usage
map_with_legend = generate_map_with_legend(annual_hot_days_count, vis_params, opacity=0.7)
map_with_legend


In [28]:
# Example: Exporting the annual hot days count image for 2015
export_image = ee.Image(annual_hot_days_count)  # Assuming this is the correct image for 2015

# Convert the image to a supported data type. Here's an example converting to Float.
export_image_converted = export_image.toFloat()

# Correctly specifying the region property using an Earth Engine Geometry object
task = ee.batch.Export.image.toDrive(**{
    'image': export_image.toFloat(),  # Assuming export_image needs to be converted
    'description': f'usa_AnnualHotDays30C_{year}',
    'folder': 'GEE_Exports',
    'fileNamePrefix': f'usa _Annual_Hot_Days30C_{year}',
    'scale': 1000,  # Adjust as necessary
    'region': usa_bbox,  # Direct use of Earth Engine Geometry object
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})

# Start the export task
task.start()

# Check the task status
print(task.status())



{'state': 'READY', 'description': 'usa_AnnualHotDays30C_2020', 'creation_timestamp_ms': 1715160313490, 'update_timestamp_ms': 1715160313490, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'NGCZOM2ISPTEJ43LKWUQSQF2', 'name': 'projects/earthengine-legacy/operations/NGCZOM2ISPTEJ43LKWUQSQF2'}


In [8]:
# Define the study area (Ethiopia's bounding box)

# Define the time range
start_date = '1979-01-01'
end_date = '1979-12-31'

# Function to filter ERA5 daily maximum temperature and check if above 35°C
def filter_temp(image):
    # Convert temperature from Kelvin to Celsius and subtract the mean to get anomalies
    temp_c = image.subtract(273.15)
    return temp_c.gt(35)

# Function to calculate the number of hot days per year
def count_hot_days(year):
    start = ee.Date(f'{year}-01-01')
    end = ee.Date(f'{year}-12-31')
    temp_dataset = ee.ImageCollection('ECMWF/ERA5/DAILY') \
        .select('mean_2m_air_temperature') \
        .filterDate(start, end) \
        .filterBounds(usa_bbox) \
        .map(filter_temp)
    # Sum the daily values to get the annual count of hot days
    sum_hot_days = temp_dataset.reduce(ee.Reducer.sum())
    return sum_hot_days.set('year', year)

# Loop through years and count hot days
years = list(range(1979, 1980))
hot_days_by_year = [count_hot_days(year) for year in years]

# Mapping function to display the hot days on a map
def map_hot_days(image, year):
    # Assuming 'image' is a single-band image representing hot days
    vis_params = {
        'min': 0,
        'max': 1,  # Adjust max if necessary, depending on the data's range
        'palette': ['white', 'red']
    }
    
    try:
        map_id_dict = image.getMapId(vis_params)
        folium_map = folium.Map(location=[9.145, 40.489673], zoom_start=6)
        folium.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map data © Google Earth Engine',
            overlay=True,
            name=f'Hot Days in {year}',
        ).add_to(folium_map)
        return folium_map
    except Exception as e:
        print(f"Error visualizing hot days for {year}: {e}")

In [ ]:
# Example usage
hot_days_2015_image = ee.Image(hot_days_by_year[5]) # Adjust index for the correct year
map_hot_days(hot_days_2015_image, 2015)